In [12]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [13]:
import numpy as np
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D
from keras.optimizers import SGD
from keras.utils import to_categorical
import pandas as pd

import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

In [14]:
labelsDF = pd.DataFrame.from_csv('/home/katya/data/stage1_labels.csv', sep=',', index_col=None)
class_weights = {0:len(labelsDF)/sum(labelsDF.cancer), 1:len(labelsDF)/len(labelsDF) - sum(labelsDF.cancer)}
del labelsDF

In [15]:
# Loading patients in the forma
def data_for_keras(number_of_patients):
    
    labelsDF = pd.DataFrame.from_csv('/home/katya/data/stage1_labels.csv', sep=',', index_col=None)

    imgs = []
    labels = []
    tempDF = labelsDF.sample(number_of_patients).reset_index()

    for i in range(len(tempDF)):
        img = np.load('/home/katya/data/processed_data_ResSeg/' + tempDF.id.ix[i] + '.npy')
        img = np.expand_dims(img,axis=0)
        imgs.append(img)
        labels.append(tempDF.cancer.ix[i])

    X = np.array(imgs)

    Y = np.array(labels)
    
    return X, Y

In [16]:
x_train, y_train = data_for_keras(1)
input_shape = x_train.shape[1:]
del x_train, y_train

In [17]:
def patientsGenerator():
    for i in range(140):
        x_train, y_train = data_for_keras(10)
        y_train = to_categorical(y_train)
        
        yield x_train, y_train

In [22]:
model = Sequential()

model.add(Conv3D(6, (3, 3, 3), activation='tanh', strides=4, data_format='channels_first', input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(3, 3, 3), strides=2, data_format='channels_first'))
model.add(Conv3D(16, (3, 3, 3), strides=1, activation='tanh'))
model.add(MaxPooling3D(pool_size=(3, 3, 3), strides=2, data_format='channels_first'))
model.add(Conv3D(120, (3, 3, 3), strides=1, activation='tanh'))

model.add(Flatten())
model.add(Dense(84, activation='tanh'))
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)
print(model.summary())

hist = model.fit_generator(patientsGenerator(), steps_per_epoch = 2, nb_epoch = 20, verbose=1, class_weight=class_weights)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 6, 33, 25, 34)     168       
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 6, 16, 12, 16)     0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 4, 14, 10, 16)     6928      
_________________________________________________________________
max_pooling3d_14 (MaxPooling (None, 4, 6, 4, 7)        0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 2, 4, 2, 120)      22800     
_________________________________________________________________
flatten_7 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 84)                161364    
__________

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=2, class_weight={0: 3.8591..., epochs=20, verbose=1)`


Epoch 1/20
2/2 [==============================] - 2s - loss: -942.0941     
Epoch 2/20
2/2 [==============================] - 1s - loss: -1163.7264     
Epoch 3/20
2/2 [==============================] - 1s - loss: -1745.5897     
Epoch 4/20
2/2 [==============================] - 1s - loss: -1745.5896    
Epoch 5/20
2/2 [==============================] - 1s - loss: -3200.2476     
Epoch 6/20
2/2 [==============================] - 1s - loss: -1454.6580     
Epoch 7/20
2/2 [==============================] - 1s - loss: -872.7948     
Epoch 8/20
2

ValueError: Error when checking model target: expected dense_14 to have shape (None, 2) but got array with shape (10, 1)

In [9]:
model.evaluate_generator(patientsGenerator(),1)

3.2236192226409912